In [1]:
import pandas as pd
import numpy as np
import bisect
import random
import math
import time
from ctypes import c_double
random.seed(0) # set specific seed for reproduction

## simulation_setup

* This script sets up the environment required for our simulation
* The output will be a Python dictionary saved at "simulated_values.pkl"
```
simulated_values.keys() == dict_keys(['simulated_time', 'simulated_distance', 'simulated_trip', 'wait_time', 'simulated_fare'])
```
**Time simulation**

- turn buffer to numpy
- avoid repetitive filtering
- try to use numba.jit but not compatible with interp1d function
- multiprocessing


**Trip simulation**

- multiprocessing

**wait time**
- turn to np.array

**Distance simulation**

- pandas group by 
- calcualte all at once by pandas apply

**Fare simulation**
- numpy broadcasting

In [158]:
!python simulation_setup.py   # simulation setup

Time took for time simulation: 11.989735126495361
Time took for trip simulation: 29.841912984848022
The R squared of the model is:  0.8705780665239197
Time took for distance simulation: 0.23627686500549316
Time took for distance & wait time simulation: 0.0191953182220459
Finish simulation setup, total time spent: 43.08929991722107


In [153]:
 !python taxi_simulation.py "one"

8:00:00 : Driver 1 start working at zone 22
8:12:00 : New trip from zone 18 to zone 18 assigned!!!
8:12:00 : New trip from zone 18 to zone 18 started!!!
8:12:00 : Trip completed at 18,Total fare = $ 10.32490214638307
8:22:00 : New trip from zone 14 to zone 14 assigned!!!
8:22:00 : New trip from zone 14 to zone 14 started!!!
8:22:00 : Trip completed at 14,Total fare = $ 15.69908694469023
8:28:00 : New trip from zone 19 to zone 19 assigned!!!
8:28:00 : New trip from zone 19 to zone 19 started!!!
8:28:00 : Trip completed at 19,Total fare = $ 24.63054626786342
8:37:00 : New trip from zone 18 to zone 18 assigned!!!
8:37:00 : New trip from zone 18 to zone 18 started!!!
8:37:00 : Trip completed at 18,Total fare = $ 35.3656760254502
8:46:00 : New trip from zone 11 to zone 11 assigned!!!
8:46:00 : New trip from zone 11 to zone 11 started!!!
8:46:00 : Trip completed at 11,Total fare = $ 46.19618987936103
8:58:00 : New trip from zone 19 to zone 19 assigned!!!
8:58:00 : New trip from zone 19 to zo

In [157]:
!python taxi_simulation.py "all_zones_once"

8:00:00 : Driver 1 start working at zone 0
12:34:00 : Rest for 30 min 
13:34:00 : New trip from zone 19 to zone 19 assigned!!!
13:34:00 : New trip from zone 19 to zone 19 started!!!
13:34:00 : Trip completed at 19,Total fare = $ 167.13332415733993
17:01:00 : Rest for 30 min 
19:03:00 : New trip from zone 8 to zone 8 assigned!!!
19:03:00 : New trip from zone 8 to zone 8 started!!!
19:03:00 : Trip completed at 8,Total fare = $ 320.4640048501854
From 8:00:00 to 22:10:00 : 59 trips finished ($ 476.9875497918956!!!)
------------------------------
8:00:00 : Driver 1 start working at zone 1
12:31:00 : Rest for 30 min 
13:49:00 : New trip from zone 23 to zone 23 assigned!!!
13:49:00 : New trip from zone 23 to zone 23 started!!!
13:49:00 : Trip completed at 23,Total fare = $ 184.5367593674182
17:04:00 : Rest for 30 min 
18:05:00 : New trip from zone 18 to zone 18 assigned!!!
18:05:00 : New trip from zone 18 to zone 18 started!!!
18:05:00 : Trip completed at 18,Total fare = $ 340.10965959442416
